In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAIfrom pydantic import BaseModel, Fieldfrom dotenv import load_dotenvfrom langgraph.graph import StateGraph, START, ENDfrom IPython.display import Image, displayload_dotenv()llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")from typing_extensions import TypedDict# Graph stateclass State(TypedDict):    topic: str    joke: str    improved_joke: str    final_joke: str# Nodesdef generate_joke(state: State):    """First LLM call to generate initial joke"""    msg = llm.invoke(f"Write a short joke about {state['topic']}")    return {"joke": msg.content}def improve_joke(state: State):    """Second LLM call to improve the joke"""    msg = llm.invoke(f"Make this joke funnier by adding wordplay: {state['joke']}")    return {"improved_joke": msg.content}def polish_joke(state: State):    """Third LLM call for final polish"""    msg = llm.invoke(f"Add a surprising twist to this joke: {state['improved_joke']}")    return {"final_joke": msg.content}# Conditional edge function to check if the joke has a punchlinedef check_punchline(state: State):    """Gate function to check if the joke has a punchline"""    # Simple check - does the joke contain "?" or "!"    if "?" in state["joke"] or "!" in state["joke"]:        return "Pass"    return "Fail"# Build workflowworkflow = StateGraph(State)# Add nodesworkflow.add_node("generate_joke", generate_joke)workflow.add_node("improve_joke", improve_joke)workflow.add_node("polish_joke", polish_joke)# Add edges to connect nodesworkflow.add_edge(START, "generate_joke")workflow.add_conditional_edges(    "generate_joke", check_punchline, {"Pass": "improve_joke", "Fail": END})workflow.add_edge("improve_joke", "polish_joke")workflow.add_edge("polish_joke", END)# Compilechain = workflow.compile()# Show workflowdisplay(Image(chain.get_graph().draw_mermaid_png()))